### Oppgave 2: Finne landene med høyeste og laveste BNP per capita
Vi tar utgangspunkt i datasettet om innbyggerantall registrert i 2020. Det finnes foreløpig bare ett datasett med oversikt over landenes BNP.

Laster inn bibliotek:

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

2.1 Finne og leses inn datasett

In [ ]:
#skriv koden for å finne og lese inn datasettene om innbyggerantall og bnp
spark.read.path("/felles/veiledning/pyspark/eksempler*").show(10, False)

df_bnp = spark.read.path("/felles/veiledning/pyspark/eksempler/bnp")
df_innbyggerantall_2020 = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")
print("*********************************** BNP ***********************************")
df_bnp.show(50,False)
print("*********************************** INNBYGGERANTALL 2020 ***********************************")
df_innbyggerantall_2020.show(50,False)

2.2 Fjerne dubletter

In [ ]:
#skriv koden for å finne og eventuelt fjerne dubletter
#Sorterer etter land og innbyggerantall og lister ut den sorterte dataframen
print("*********************************** INNBYGGERANTALL SORTERT ETTER LAND OG INNBYGGERANTALL ***********************************")

df_innbyggerantall_2020_sort = df_innbyggerantall_2020.orderBy(['Land','Innbyggerantall'], ascending=[1, 0])
df_innbyggerantall_2020_sort.show(50,False)


#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_land_dub_alle  = df_innbyggerantall_2020_sort.join(df_innbyggerantall_2020_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')
print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_land_dub_alle.show()  

print("*********************************** INNBYGGERANTALL UTEN DUBLIKATER. HØYESTE INNBYGGERANTALL ER BEHOLDT ***********************************")
df_innbyggerantall_udub= df_innbyggerantall_2020_sort.drop_duplicates(subset=['Land'])
df_innbyggerantall_udub.show(50,False)


In [ ]:
#skriv koden for å finne og eventuelt fjerne dubletter
#Sorterer etter land og innbyggerantall og lister ut den sorterte dataframen
print("*********************************** BNP SORTERT ETTER LAND OG STØRRELSE PÅ BNP ***********************************")

df_bnp_sort = df_bnp.orderBy(['Land','BNP'])
df_bnp_sort.show(50,False)


#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_bnp_dub_alle  = df_bnp_sort.join(df_bnp_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')
print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_bnp_dub_alle.show()  


2.3 Koble datasett

In [ ]:
#skriv koden for å koble datasettene
df_bnp_innbyggerantall = df_bnp.join(df_innbyggerantall_udub, 'Land', 'inner')
print("*********************************** SAMMENKOBLET DATASETT ***********************************")
df_bnp_innbyggerantall.show(50,False)

2.4 Lage en ny variabel

In [ ]:
#skriv koden for å lage ny variabel (BNP/innbyggerantall)
df_bnp_innbyggerantall = df_bnp_innbyggerantall.withColumn('BNP per capita',df_bnp_innbyggerantall['BNP']/df_bnp_innbyggerantall['Innbyggerantall'])
print("*********************************** SAMMENKOBLET DATASETT MED NY VARIABEL ***********************************")
df_bnp_innbyggerantall.show(50,False)

2.5 Finne min og maks

In [ ]:
#skriv koden for å finne min, maks og gjennomsnitt av den nye variabelen

from pyspark.sql.functions import mean, min, max
result = df_bnp_innbyggerantall.select([mean("BNP per capita"), min("BNP per capita"), max("BNP per capita")])
print("*********************************** Minimum, maksimum og gjennomsnitt av BNP per capita ***********************************")
result.show()
